In [1]:
#import packages 
import pandas as pd 
import geopandas
import geoplot 
import codecs
import osm2geojson
import json
import csv 
import os
import random
import numpy as np
from datetime import datetime
import time
from collections import defaultdict
import gzip
import csv
import pickle

In [ ]:
#This workbook includes an outline of code (and code snippets) that will be used to populate an URBANopt GeoJSON input file. 
#The process beings with an OpenStreetMap OSM file for a given district. The following steps are then followed to 
#populate GeoJSON fileds required by URBANopt. 

#1. Create GeoJSON from OSM. 

#2. Populate GeoJSON fields required by URBANopt based on inferences, or look-ups from external data sources (including a 
#spreadsheet that has been populated wiht building metadata.)

#3. Use distributions available from other sources to infer attributes (such as HVAC system types) based on high-level information
#regarding the building location, type, and vintage when information specific to the building is not available. 

#Note that some code shown here has not been fully adapted to the exact input files, but is included as a proof of concept.  

In [ ]:
#Set filepaths
base_filepath='D:\\BlocPower_BTO\\'
OSM_filename='Southside_Ithaca.osm'

In [ ]:
#Process OSM into GeoJSON (from OSM2GeoJSON package documentation)
input_OSM_filepath=base_filepath + OSM_filename #input file from OSM 

with codecs.open(input_OSM_filepath, 'r', encoding='utf-8') as data:
    xml = data.read()

geojson = osm2geojson.xml2geojson(xml)

with open('data_r.json','w') as f:
    json.dump(geojson, f)

In [ ]:
#Select only polygons from the GeoJSON (to remove additional node markers)
data = gpd.read_file("data_r.json")
data_bldgs = data.loc[data.type == 'Polygon']

#Write to output file
data_bldgs.to_file('output_geojson.json', driver="GeoJSON")  

In [ ]:
#Format and augment GeoJSON for URBANopt 
#Populate required fields 

#Building_type:
#Map "amenity" field in GeoJSON to the available URBANopt building types (see block below for details)

#Number of stories:
#Populate from data spreadsheet, looking up based on street address

#Number of stories above ground:
#Populate from data spreadsheet, looking up based on street address

#Template (energy code version):
#Infer based on year of construction, which is looked up from data spreadsheet based on street address

#Year built:
#Populate from data spreadsheet, looking up based on street address

#Heating system fuel type: 
#Populate from data spreadsheet, looking up based on street address

#Water heating fuel type:
#Populate from data spreadsheet, looking up based on street address

#HVAC system type 
#Residential bldgs, heating: 
#Populate from data spreadsheet, looking up based on street address, and using inferences
#Hot-water/steam=>Natural gas boiler 
#Air-based =>Natural gas furnace

#Residential bldgs, cooling: 
#Populate from data spreadsheet, looking up based on street address
#If central cooling identified, infer a split-system DX 
#If identified as having no central cooling, use ResStock distributions to infer the presence of a window unit 

#Commercial buidlings, heating and cooling 
#If stated to be fully cooled (infer based on ASHRAE 90.1 Appendix G)
#If 3 floors or fewer and <25,000 ft2, infer packaged single zone systems with DX cooling and gas heating
#If 4 or 5 floors and <25,000 ft2 or 5 floors or fewer and 25,000-150,000 ft2, infer a packaged VAV with reheat 
#If stated to have no cooling, identify as no cooling 

#Alternative approach: Use distributions from ComStock or another source to infer system types based on building size, vintage
#and location 

#Fields to be automatically populated by URBANopt
#Footprint area, perimeter, total area 

In [ ]:
#Example code for assigning a decade of construction based on a year built, and assigning heating fuel and HVAC distribution system types
def assign_htg_dist(heat_sys): #deal with case
    if pd.isna(heat_sys):
        return 'NA'
    if "water" in heat_sys:
        return 'Non_Ducted'
    if heat_sys == 'hot air': 
         return 'Ducted'

def assign_htg_fuel(Fuel_Type):
    if pd.isna(Fuel_Type):
        return 'NA'
    if 'Gas' in Fuel_Type:
        return 'Natural Gas'
def assign_decade(row):
    if row == 'ND' or row =='' or row=='NL':
        return 'NA'
    if row < 1940:
        return '<1940'
    else:
        return (str(row)+'s')

assessment_data['Decade_built']=assessment_data.apply(lambda row: (assign_decade(row.Eff_year_built)),axis=1) 
assessment_data['Heating_Dist']=assessment_data.apply(lambda row: (assign_htg_dist(row.Heat_Type)),axis=1) 
assessment_data['Htg_Fuel']=assessment_data.apply(lambda row: (assign_htg_fuel(row.Fuel_Type)),axis=1) 

In [ ]:
#Example code for looking up a value from a spreadsheet based on a key in a GeoJSON file, and adding a field value to the GeoJSON 
#file with that value

def read_data(geojson_file): #Load a GeoJSON file 
     f = open(geojson_file, "r")   
     geojson=json.load(f)
     f.close()
     return geojson  

#This method modifies a GeoJSON file (the 'input GeoJSON file') to set system types of features by an id number, using data from the 'input data' file 
def process_item(ident, sys_type, geojson_file):  
    system=sys_type   
    geojson=read_data(geojson_file) 
    bldg= [obj for obj in geojson['features'] if obj['properties']['LOCADDR']==(ident)] #ObjectID is an identifier from the Tompkins County GeoJSON data 
    if bldg: #make sure list isn't empty 
        bldg[0]['properties']['system_type']=system 
        f = open(geojson_file, "w")
        json.dump(geojson, f, indent=4, sort_keys=True) #this formats the output GeoJSON 
        f.close()
    else:
        print("ID not present in GeoJSON file") 

In [ ]:
#Example Ruby code for inferring HVAC system type (Rawad El-Kontar)

def add_system_type(geojson_file_name)

  # read geojson
  initial_geojson_path = File.join(File.dirname(__FILE__), geojson_file_name)
  initial_geojson = JSON.parse(File.read(File.absolute_path(initial_geojson_path)))

  initial_geojson["features"].each do |feature|


    if feature["properties"]["type"] == "Building"
      # get: floor area , number of stories , building type
      floor_area = feature["properties"]["floor_area"]
      number_of_stories = feature["properties"]["number_of_stories"]
      building_type = feature["properties"]["building_type"]

      # define list of residential
      lst = ['Single-Family', 'Multifamily (2 to 4 units)','Multifamily (5 or more units)']
      
      # define mixed used bldgs types 
      mixed_type_1 = feature["properties"]["mixed_type_1"]
      mixed_type_2 = feature["properties"]["mixed_type_2"]


      # define system_type 
      if mixed_type_1 

        if mixed_type_1.include? 'Multifamily'
          system_type = "PTHP"
        elsif mixed_type_2.include? 'Multifamily'
          system_type = "PTHP"
        else 
          if number_of_stories <= 3 && floor_area < 25000
            system_type = "PSZ-HP"
          elsif number_of_stories <= 5 && floor_area <= 150000 && floor_area >= 25000
            system_type = "PVAV with PFP boxes"
          else 
            system_type = "VAV chiller with PFP boxes"
          end
        end
    
      else

        if lst.any? { |s| building_type.include? s } 
          system_type = "PTHP"
        elsif number_of_stories <= 3 && floor_area < 25000
          system_type = "PSZ-HP"
        elsif number_of_stories <= 5 && floor_area <= 150000 && floor_area >= 25000
          system_type = "PVAV with PFP boxes"
        else 
          system_type = "VAV chiller with PFP boxes"
        end

      end

      feature["properties"].merge!("system_type": system_type)

      #puts feature["properties"]
      
    end


  end

In [ ]:
#Proposed assignment scheme for bldg types


#1-family res => Single-family detached
#2-family res, 3-family res =>Multifamily 
# Apartment building => Multi-family 2-4 units or Multifamily 5 or more units 
#Professional building => Office 
#Restaurant => Food service 
#Attached row building =>mixed use, often with office and retail spaces or strip shopping mall 
#Office = > office
#One-use small building => office or retail, depending on use 
#Inn/lodge => Lodging 
#Mini mart =>retail other than mall 



#Residential building types in URBANopt
'Single-Family Detached',
'Single-Family Attached',
'Multifamily'


#Commercial building types in URBANopt
'Vacant',
'Office',
'Laboratory',
'Nonrefrigerated warehouse',
'Food sales',
'Public order and safety',
'Outpatient health care',
'Refrigerated warehouse',
'Religious worship',
'Public assembly',
'Education',
'Food service',
'Inpatient health care',
'Nursing',
'Lodging',          
'Strip shopping mall',
'Enclosed mall',
'Retail other than mall',
'Service',
'Uncovered Parking',
'Covered Parking',
'Mixed use',
'Multifamily (2 to 4 units)',
'Multifamily (5 or more units)',
'Single-Family'


In [ ]:
#Proposed look-up scheme for energy code template, based on "effective year of construction" (the year of the most recent
#major renovation, if one has taken place)

#Pre-1980=>#DOE Ref Pre 1980

#1980-2008: #DOE Ref 1980-2004
#2008-2010 => #90.1 2004 (Based on NYS adoption of ASHRAE 90.1 2004 in 2008)
#2010-2016 => #90.1 2007 (NYS seemed to have adopted a code similar to 90.1 2007 around 2010)

#2016-present: #90.1 2013 (Based on NYS adoption of ASHRAE 90.1 2013 as of 2016)

#A 90.1 2010 template is also available in URBANopt, but it's not clear that a code similar to that was ever used in New York. 
#Reference for NYS code adoption: https://www.energycodes.gov/status/states/new-york

In [ ]:
#Possible future work for further refinement

#Schedules accounting for stochasticity in occupancy, plug load use, etc 
#Example code for incorporating stochasticity in schedules (Rawad El-Kontar and Jing Wang)

# problist
###times_lsts
wkdy_start_times_lst = defaultdict(bool)
wkdy_oper_hrs_lst = defaultdict(bool)
wknd_start_times_lst = defaultdict(bool)
wknd_oper_hrs_lst = defaultdict(bool)

###prob
wkdy_start_times_prob = defaultdict(bool)
wkdy_oper_hrs_prob = defaultdict(bool)
wknd_start_times_prob = defaultdict(bool)
wknd_oper_hrs_prob = defaultdict(bool)


    
bldg_types = ['out_hospital','mall', 'office', 'pr_school', 'restaurant', 'sc_school','supermarket','hotel'] 

for bldg_type in bldg_types:
    
    #### read the start time and operation hrs from the prob_dist csv files
    
    # wkdy start time
    wkdy_start_times_csv_path = os.path.join('results','com_occ_pro_dist', bldg_type + '_wkdy_start_times_dist.csv')
    df_wkdy_start_times = pd.read_csv(wkdy_start_times_csv_path, sep=',\s+', delimiter=',', encoding="utf-8",\
                    skipinitialspace=True)
    #remove 24hrs and closed rows 
    df_wkdy_start_times = df_wkdy_start_times[~df_wkdy_start_times['start_hrs'].isin(['24hrs', 'closed'])]
    
    
    # wkdy_oper_hrs
    wkdy_oper_hrs_csv_path = os.path.join('results','com_occ_pro_dist', bldg_type + '_wkdy_oper_hrs_dist.csv')
    df_wkdy_oper_hrs = pd.read_csv(wkdy_oper_hrs_csv_path, sep=',\s+', delimiter=',', encoding="utf-8",\
                    skipinitialspace=True)
        #replace 24hrs with 24:00 and closed with 0:00
    df_wkdy_oper_hrs['operational_hrs'] = df_wkdy_oper_hrs['operational_hrs'].str.replace("24hrs", "24:00")  
    df_wkdy_oper_hrs['operational_hrs'] = df_wkdy_oper_hrs['operational_hrs'].str.replace("closed", "00:00") 
    
    # wknd start time
    wknd_start_times_csv_path = os.path.join('results','com_occ_pro_dist', bldg_type + '_wknd_start_times_dist.csv')
    df_wknd_start_times = pd.read_csv(wknd_start_times_csv_path, sep=',\s+', delimiter=',', encoding="utf-8",\
                    skipinitialspace=True)  
    #remove 24hrs and closed rows 
    df_wknd_start_times = df_wknd_start_times[~df_wknd_start_times['start_hrs'].isin(['24hrs', 'closed'])]
    
    # wknd_oper_hrs
    wknd_oper_hrs_csv_path = os.path.join('results','com_occ_pro_dist', bldg_type + '_wknd_oper_hrs_dist.csv')
    df_wknd_oper_hrs = pd.read_csv(wknd_oper_hrs_csv_path, sep=',\s+', delimiter=',', encoding="utf-8",\
                    skipinitialspace=True)
    #replace 24hrs with 24:00 and closed with 0:00
    df_wknd_oper_hrs['operational_hrs'] = df_wknd_oper_hrs['operational_hrs'].str.replace("24hrs", "24:00")  
    df_wknd_oper_hrs['operational_hrs'] = df_wknd_oper_hrs['operational_hrs'].str.replace("closed", "00:00")    
    
    ###times_lst
    wkdy_start_times_lst[bldg_type]= df_wkdy_start_times['start_hrs']
    wkdy_oper_hrs_lst[bldg_type] = df_wkdy_oper_hrs['operational_hrs']
    wknd_start_times_lst[bldg_type] = df_wknd_start_times['start_hrs']
    wknd_oper_hrs_lst[bldg_type] = df_wknd_oper_hrs['operational_hrs']
    ###prob
    wkdy_start_times_prob[bldg_type] = df_wkdy_start_times['probability']
    wkdy_oper_hrs_prob[bldg_type] = df_wkdy_oper_hrs['probability']
    wknd_start_times_prob[bldg_type] = df_wknd_start_times['probability']
    wknd_oper_hrs_prob[bldg_type] = df_wknd_oper_hrs['probability']

#%% load geojson file
path=os.getcwd()
with open(os.path.join(path, 'penastation_0629.json')) as f:
    geojson=json.load(f)

##lsts
wkdy_start_times_lst
wkdy_oper_hrs_lst
wknd_start_times_lst
wknd_oper_hrs_lst
###prob
wkdy_start_times_prob
wkdy_oper_hrs_prob
wknd_start_times_prob
wknd_oper_hrs_prob

['out_hospital','mall', 'office', 'pr_school', 'restaurant', 'sc_school','supermarket'] 

#%% for each building feature, add new properties
# Note: Commercial only
for key in geojson['features']:
    if key['properties']['type'] == 'Building':

        if 'Office' in key['properties']['building_type']:
            #weekday
            wkdy_start = np.random.choice(wkdy_start_times_lst['office'], p=wkdy_start_times_prob['office'])
            wkdy_oper_hrs = np.random.choice(wkdy_oper_hrs_lst['office'], p=wkdy_oper_hrs_prob['office'])    
            #weekend
            wknd_start = np.random.choice(wknd_start_times_lst['office'], p=wknd_start_times_prob['office'])
            wknd_oper_hrs = np.random.choice(wknd_oper_hrs_lst['office'], p=wknd_oper_hrs_prob['office'])
            ### add values in geojson file
            #weekday
            key['properties']['weekday_start_time'] = wkdy_start
            key['properties']['weekday_duration'] = wkdy_oper_hrs
            #weekend
            key['properties']['weekend_start_time'] = wknd_start
            key['properties']['weekend_duration'] = wknd_oper_hrs

#Envelope construction

In [ ]:
#For reference: URBANopt HVAC system types

#Residential

Air to air heat pump, mini-split heat pump, ground-to-air heat pump
Electric resistance, furnace (natural gas, fuel oil, electricity, wood, propane)
Boiler (electric, natural gas, fuel oil, wood, propane)
Central air-conditioner
Room air-conditioner 


#Commercial 

'Baseboard electric'
'Baseboard gas boiler'
'Baseboard central air source heat pump'
'Baseboard district hot water'
'Direct evap coolers with baseboard electric'
'Direct evap coolers with baseboard gas boiler'
'Direct evap coolers with baseboard central air source heat pump'
'Direct evap coolers with baseboard district hot water'
'Direct evap coolers with forced air furnace'
'Direct evap coolers with gas unit heaters'
'Direct evap coolers with no heat'
'DOAS with fan coil chiller with boiler'
'DOAS with fan coil chiller with central air source heat pump'
'DOAS with fan coil chiller with district hot water'
'DOAS with fan coil chiller with baseboard electric'
'DOAS with fan coil chiller with gas unit heaters'
'DOAS with fan coil chiller with no heat'
'DOAS with fan coil air-cooled chiller with boiler'
'DOAS with fan coil air-cooled chiller with central air source heat pump'
'DOAS with fan coil air-cooled chiller with district hot water'
'DOAS with fan coil air-cooled chiller with baseboard electric'
'DOAS with fan coil air-cooled chiller with gas unit heaters'
'DOAS with fan coil air-cooled chiller with no heat'
'DOAS with fan coil district chilled water with boiler'
'DOAS with fan coil district chilled water with central air source heat pump'
'DOAS with fan coil district chilled water with district hot water'
'DOAS with fan coil district chilled water with baseboard electric'
'DOAS with fan coil district chilled water with gas unit heaters'
'DOAS with fan coil district chilled water with no heat'
'DOAS with VRF'
'DOAS with water source heat pumps fluid cooler with boiler'
'DOAS with water source heat pumps cooling tower with boiler'
'DOAS with water source heat pumps with ground source heat pump'
'DOAS with water source heat pumps district chilled water with district hot water'
'Fan coil chiller with boiler'
'Fan coil chiller with central air source heat pump'
'Fan coil chiller with district hot water'
'Fan coil chiller with baseboard electric'
'Fan coil chiller with gas unit heaters'
'Fan coil chiller with no heat'
'Fan coil air-cooled chiller with boiler'
'Fan coil air-cooled chiller with central air source heat pump'
'Fan coil air-cooled chiller with district hot water'
'Fan coil air-cooled chiller with baseboard electric'
'Fan coil air-cooled chiller with gas unit heaters'
'Fan coil air-cooled chiller with no heat'
'Fan coil district chilled water with boiler'
'Fan coil district chilled water with central air source heat pump'
'Fan coil district chilled water with district hot water'
'Fan coil district chilled water with baseboard electric'
'Fan coil district chilled water with gas unit heaters'
'Fan coil district chilled water with no heat'
'Forced air furnace'
'Gas unit heaters'
'PTAC with baseboard electric'
'PTAC with baseboard gas boiler'
'PTAC with baseboard district hot water'
'PTAC with gas unit heaters'
'PTAC with electric coil'
'PTAC with gas coil'
'PTAC with gas boiler'
'PTAC with central air source heat pump'
'PTAC with district hot water'
'PTAC with no heat'
'PTHP'
'PSZ-AC with baseboard electric'
'PSZ-AC with baseboard gas boiler'
'PSZ-AC with baseboard district hot water'
'PSZ-AC with gas unit heaters'
'PSZ-AC with electric coil'
'PSZ-AC with gas coil'
'PSZ-AC with gas boiler'
'PSZ-AC with central air source heat pump'
'PSZ-AC with district hot water'
'PSZ-AC with no heat'
'PSZ-AC district chilled water with baseboard electric'
'PSZ-AC district chilled water with baseboard gas boiler'
'PSZ-AC district chilled water with baseboard district hot water'
'PSZ-AC district chilled water with gas unit heaters'
'PSZ-AC district chilled water with electric coil'
'PSZ-AC district chilled water with gas coil'
'PSZ-AC district chilled water with gas boiler'
'PSZ-AC district chilled water with central air source heat pump'
'PSZ-AC district chilled water with district hot water'
'PSZ-AC district chilled water with no heat'
'PSZ-HP'
'PVAV with gas boiler reheat'
'PVAV with central air source heat pump reheat'
'PVAV with district hot water reheat'
'PVAV with PFP boxes'
'PVAV with gas heat with electric reheat'
'Residential AC with baseboard electric'
'Residential AC with baseboard gas boiler'
'Residential AC with baseboard central air source heat pump'
'Residential AC with baseboard district hot water'
'Residential AC with residential forced air furnace'
'Residential AC with no heat'
'Residential heat pump'
'Residential heat pump with no cooling'
'Residential forced air furnace'
'VAV chiller with gas boiler reheat'
'VAV chiller with central air source heat pump reheat'
'VAV chiller with district hot water reheat'
'VAV chiller with PFP boxes'
'VAV chiller with gas coil reheat'
'VAV chiller with no reheat with baseboard electric'
'VAV chiller with no reheat with gas unit heaters'
'VAV chiller with no reheat with zone heat pump'
'VAV air-cooled chiller with gas boiler reheat'
'VAV air-cooled chiller with central air source heat pump reheat'
'VAV air-cooled chiller with district hot water reheat'
'VAV air-cooled chiller with PFP boxes'
'VAV air-cooled chiller with gas coil reheat'
'VAV air-cooled chiller with no reheat with baseboard electric'
'VAV air-cooled chiller with no reheat with gas unit heaters'
'VAV air-cooled chiller with no reheat with zone heat pump'
'VAV district chilled water with gas boiler reheat'
'VAV district chilled water with central air source heat pump reheat'
'VAV district chilled water with district hot water reheat'
'VAV district chilled water with PFP boxes'
'VAV district chilled water with gas coil reheat'
'VAV district chilled water with no reheat with baseboard electric'
'VAV district chilled water with no reheat with gas unit heaters'
'VAV district chilled water with no reheat with zone heat pump'
'VRF'
'Water source heat pumps fluid cooler with boiler'
'Water source heat pumps cooling tower with boiler'
'Water source heat pumps with ground source heat pump'
'Water source heat pumps district chilled water with district hot water'
'Window AC with baseboard electric'
'Window AC with baseboard gas boiler'
'Window AC with baseboard central air source heat pump'
'Window AC with baseboard district hot water'
'Window AC with forced air furnace'
'Window AC with unit heaters'
'Window AC with no heat'